In [1]:
import pandas as pd
from Acesso import Login
from Query import Query
from glob import glob
from datetime import datetime

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

pd.set_option('display.max_columns',None)

In [2]:
querys={
    
    'Margem':
    
    """
    
    SELECT * FROM netfeira.vw_margens
    
    """,
    
    'Cliente':
    
    """
    
    SELECT * FROM netfeira.vw_cliente
    
    """
}

In [3]:
tabelas_df=sql.CriarTabela(kwargs=querys)

In [56]:
tabelas_df['Consolidado']=tabelas_df['Margem'].loc[(tabelas_df['Margem']['Tipo de Operação']=='VENDAS')]

In [57]:
tabelas_df['Consolidado']=tabelas_df['Consolidado'].merge(tabelas_df['Cliente'][['ID Cliente','Razão Social','Nome Fantasia','Desc Financeiro']],on='ID Cliente',how='inner')

In [58]:
tabelas_df['Consolidado'].columns

Index(['Origem', 'Situação Entrega', 'ID Situação', 'Situação', 'Pedido',
       'NFe', 'Tipo de Pedido', 'Tipo de Operação', 'Tabelas',
       'Data de Emissão', 'Data de Faturamento', 'Data de Entrega', 'Empresa',
       'ID Cliente', 'ID Vendedor', 'SKU', 'Seq', 'Qtde', 'Unid. VDA', 'Fator',
       'Qtde. VDA', 'Valor VDA', 'Total Venda', 'Total AV', 'MG CRP',
       'Margem CUE', 'Margem CMP', 'Margem CRP', 'MG CUE', 'MG CMP',
       'Comsissão R$', 'COFINS R$', 'PIS R$', 'ICMS R$', 'ICMS ST R$',
       'IPI R$', 'Peso Bruto KG', 'Peso Líquido KG', 'Total Geral',
       'Custo CUE', 'Custo CUE Capado', 'Razão Social', 'Nome Fantasia',
       'Desc Financeiro'],
      dtype='object')

In [59]:
ano=datetime.now().year

#mes=datetime.now().month

mes=10

dia=datetime.now().day

In [60]:
tabelas_df['Consolidado']=tabelas_df['Consolidado'].loc[(tabelas_df['Consolidado']['Data de Faturamento'].dt.year==ano)&(tabelas_df['Consolidado']['Data de Faturamento'].dt.month==mes)]

In [61]:
tabelas_df['Consolidado']['Total Financeiro']=tabelas_df['Consolidado'].apply(lambda info: round(info['Total Venda']*(1-info['Desc Financeiro']),2),axis=1)

In [62]:
tabelas_df['Consolidado'].loc[tabelas_df['Consolidado']['Desc Financeiro'].isnull(),'Total Financeiro']=tabelas_df['Consolidado']['Total Venda'].loc[tabelas_df['Consolidado']['Desc Financeiro'].isnull()]

In [63]:
tabelas_df['Consolidado']['Custo Financeiro']=tabelas_df['Consolidado'].apply(lambda info: info['Total Financeiro']-info['Custo CUE'],axis=1)

In [64]:
total=tabelas_df['Consolidado']['Total Venda'].sum()

custo_cue=tabelas_df['Consolidado']['Margem CUE'].sum()

round(custo_cue/total,4)*100

32.42

In [65]:
total=tabelas_df['Consolidado']['Total Financeiro'].sum()

custo_cue=tabelas_df['Consolidado']['Custo Financeiro'].sum()

round(custo_cue/total,4)*100

30.17

In [66]:
tabelas_df['Dados']=tabelas_df['Consolidado'][['Tabelas','Total Venda','Margem CUE','Total Financeiro','Custo Financeiro']].loc[tabelas_df['Consolidado']['Data de Faturamento'].dt.day==dia].groupby(['Tabelas'],as_index=False).sum()

In [67]:
tabelas_df['Dados']['MG CUE']=tabelas_df['Dados'].apply(lambda info: round(info['Margem CUE']/info['Total Venda'],4)*100,axis=1)

In [68]:
tabelas_df['Dados']

,Tabelas,Total Venda,Margem CUE,Total Financeiro,Custo Financeiro,MG CUE
0,DISTRIBUIDORES,3054.48,870.60,3054.48,870.60,28.50
1,PASTORINHO,3614.70,1194.28,3506.27,1085.85,33.04
2,SONDA,7618.06,2834.05,6681.03,1897.02,37.20
3,ST MARCHE,791.16,260.85,751.60,221.29,32.97
4,TABELA CONSUMIDOR FINAL,826.27,381.30,826.27,381.30,46.15
5,TABELA DE NEGOCIAÇÃO,1996.00,389.87,1996.00,389.87,19.53
6,TABELA DE PREÇO,161947.31,58917.09,161750.95,58720.73,36.38
7,TABELA FUNCIONÁRIO,101.57,9.24,101.57,9.24,9.10
8,TABELA OBA,518.00,-20.00,518.00,-20.00,-3.86
